In [2]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pydantic import BaseModel, Field, field_validator

In [22]:
class GoogleSheetToSQLiteConverter:
    def __init__(self, spreadsheet: str, credentials_path: str):
        scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        self.spreadsheet = spreadsheet
        self.client = gspread.authorize(creds)
        
    def get_sheet_data(self, sheet: str):
        worksheet = self.client.open(self.spreadsheet).worksheet(sheet)
        records = worksheet.get_all_values()

        return records

In [23]:
ghconv = GoogleSheetToSQLiteConverter(
    spreadsheet='Famaga Knowledge Map', 
    credentials_path=r'langchain-400510-06936d0d30b5.json')

sheet_data = ghconv.get_sheet_data('Discount -> Purchased same product')



In [ ]:
 = 'Discount -> Purchased same product'

In [ ]:
sheet = ghconv.client.open("Your_Sheet_Title").sheet1

# Get all values in the sheet
data = sheet.get_all_values()
headers = data.pop(0)

In [26]:
import pandas as pd


headers = sheet_data.pop(0)

# Convert to a DataFrame
df = pd.DataFrame(sheet_data, columns=headers)

# Print DataFrame
df

,Is margin still above 10%?,Is the same margin on product as previous?,instruction
0,If you change price that would be equal at pre...,Wether price or margin on the same product tha...,
1,TRUE,TRUE,Give 2% discount
2,FALSE,ANY,Ask customer justification
3,FALSE,TRUE,It seems the price cannot be the same as previ...


In [25]:
markdown_table = df.to_markdown(index=False)
print(markdown_table)

| Is margin still above 10%?   | Is the same margin on product as previous?   | instruction                                                                                                                 |
|:-----------------------------|:---------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------|
| TRUE                         | TRUE                                         | Give 2% discount                                                                                                            |
| FALSE                        | ANY                                          | Ask customer justification                                                                                                  |
| FALSE                        | TRUE                                         | It seems the price cannot be the same as previous on this product, due to some negotiation on ma

In [24]:
sheet_data

[['Is margin still above 10%?',
  'Is the same margin on product as previous?',
  'instruction'],
 ['If you change price that would be equal at previous order is our margin steel be great than 10%?',
  'Wether price or margin on the same product that have been bought earlier?',
  ''],
 ['TRUE', 'TRUE', 'Give 2% discount'],
 ['FALSE', 'ANY', 'Ask customer justification'],
 ['FALSE',
  'TRUE',
  'It seems the price cannot be the same as previous on this product, due to some negotiation on market or due to exchang rate']]

In [21]:
headers

{'Is margin still above 10%?': 'If you change price that would be equal at previous order is our margin steel be great than 10%?',
 'Is the same margin on product as previous?': 'Wether price or margin on the same product that have been bought earlier?',
 'instruction': ''}

In [27]:
def get_instruction(df, margin_above_10, same_margin):
    # Build the query string
    query_string = f'`Is margin still above 10%?` == "{margin_above_10}" and `Is the same margin on product as previous?` == "{same_margin}"'
    result = df.query(query_string)
    return result['instruction'].iloc[0]

# Example Usage
instruction = get_instruction(df, "TRUE", "TRUE")


In [28]:
instruction

'Give 2% discount'

In [30]:
from clients.gpt_database_logger import GPTDatabaseLogger

GPT_DB_LOGGER_PATH = 'sqlite:///prompt_versions.db'

db_logger = GPTDatabaseLogger(GPT_DB_LOGGER_PATH)

In [32]:
response = db_logger.create_completion(temperature=0.1, tags='query_builder', model='gpt-4', messages=[
        { "role": "user", "content": f"""
Build query for this input data.

[CLIENT PURCHASE HISTORY]
**Client purchase history:**
Date: 2020-08-25
135614 (GPC-040 Aventics (brand of Emerson)) margin: 20%, sell: 73.97$ qty. 1
135614 (1827003935 Aventics (brand of Emerson)) margin: 20%, sell: 61.01$ qty. 1
135614 (Shipping cost None) margin: 20%, sell: 72$ qty. 1

Date: 2021-06-30
190259 (058912610 Technotrans) margin: 20%, sell: 10.84$ qty. 50

Date: 2024-04-29
376924 (700Z Tomoe Valve) margin: 28%, sell: 374.27$ qty. 1

[/CLIENT PURCHASE HISTORY]

Offer:
(700Z Tomoe Valve) margin: 28%, sell: 374.27$ qty. 1
        
### Querying Instructions:

#### 1. **Field Names**:
   - **"Is margin still above 10%?"**: This field checks if the margin is above 10%.
   - **"Is the same margin on product as previous?"**: This field checks if the current margin on the product matches the previous margin.

#### 2. **Possible Values**:
   - **TRUE**: The condition is met.
   - **FALSE**: The condition is not met.
   - **ANY**: The condition is not relevant or any value is acceptable.

#### 3. **Building a Query**:
   - Use only field names that listed at instruction with possible values
   - Use the `query()` method in Pandas to filter the table based on the conditions.
   - Format each condition as a string using the field names and their desired values.

#### 5. **Example Queries**:

```json
{{
    "query": "`Is margin still above 10%?` == 'TRUE' and `Is the same margin on product as previous?` == 'TRUE'"
}}
```
"""}])

```json
{
    "query": "`Is margin still above 10%?` == 'TRUE' and `Is the same margin on product as previous?` == 'TRUE' and `Product` == '700Z Tomoe Valve'"
}
```

--------------------

Note saved without feedback. ID: 9a39d255-38f8-4b7f-a569-de7dc6507726
Input tokens: 431 Output tokens: 52 Total price: 0.02$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [34]:
query_string = "`Is margin still above 10%?` == 'TRUE' and `Is the same margin on product as previous?` == 'TRUE'"
result = df.query(query_string)
result['instruction'].iloc[0]

'Give 2% discount'